In [1]:
#scrapping santé mentale
import requests
import pandas as pd
import io

# ---------------------------------------------------------
# CONFIGURATION
# ---------------------------------------------------------
# L'URL de base pour les exports CSV des graphiques Our World in Data.
# Le "slug" correspond à la fin de l'URL de la page du graphique sur leur site.
BASE_URL = "https://ourworldindata.org/grapher/"
CHART_SLUG = "burden-of-disease-from-each-category-of-mental-illness"

def get_owid_data(slug):
    """
    Récupère le fichier CSV complet associé à un graphique OWID.
    --> Correspond à la partie 'Interroger une API / Récupérer la donnée brute'.
    """
    # Construction de l'URL (OWID expose les CSV via /grapher/slug.csv)
    url = f"{BASE_URL}{slug}.csv"
    
    print(f"Requête vers : {url}")
    
    # Envoi de la requête HTTP GET
    resp = requests.get(url)
    
    # Vérification des erreurs HTTP (404, 500, etc.)
    resp.raise_for_status()
    
    # Conversion du contenu CSV (texte) en DataFrame pandas.
    # On utilise io.StringIO car pd.read_csv attend un fichier ou un buffer, pas une simple string.
    return pd.read_csv(io.StringIO(resp.content.decode('utf-8')))

def process_mental_health_data(df, target_year=2021):
    """
    Nettoie et filtre les données pour ne garder que l'année cible.
    --> Correspond à la partie transformation et nettoyage.
    """
    
    # 1. Filtrage temporel : on ne veut que l'année 2021
    df_filtered = df[df['Year'] == target_year].copy()
    
    # 2. Nettoyage : On supprime la colonne 'Year' car elle est maintenant constante
    # On garde 'Entity' (Pays), 'Code' (Code ISO, utile pour les jointures), et les maladies.
    df_filtered = df_filtered.drop(columns=['Year'])
    
    # 3. Renommage des colonnes pour faciliter l'analyse statistique (éviter les espaces et parenthèses)
    # Exemple : "Depressive disorders (DALYs)" -> "Depressive_disorders"
    # Cela rendra l'écriture de tes formules de régression plus simple (ex: smf.ols('Y ~ X'))
    df_filtered.columns = [
        col.replace(' (DALYs)', '')       # Enlever l'unité
           .replace(' ', '_')             # Remplacer espaces par _
           .replace('-', '_')             # Remplacer tirets par _
        for col in df_filtered.columns
    ]
    
    # 4. Gestion des entités non-pays (OWID inclut des régions comme "World", "Africa", etc.)
    # Si tu ne veux que les pays qui ont un Code ISO (pour exclure les agrégats régionaux) :
    df_countries_only = df_filtered.dropna(subset=['Code'])
    
    return df_countries_only

# ---------------------------------------------------------
# PARTIE PRINCIPALE (MAIN)
# ---------------------------------------------------------

if __name__ == "__main__":
    
    print("--- Démarrage du scraping OWID ---")

    # 1. Extraction (Scraping)
    # Contrairement à l'OMS, on reçoit tout en un seul morceau.
    raw_df = get_owid_data(CHART_SLUG)
    print(f"Données brutes récupérées : {raw_df.shape[0]} lignes.")

    # 2. Transformation
    clean_df = process_mental_health_data(raw_df, target_year=2021)
    print(f"Données filtrées (2021, pays uniquement) : {clean_df.shape[0]} lignes.")
    
    # Aperçu des colonnes disponibles pour tes régressions
    print("\nColonnes disponibles pour l'analyse :")
    print(clean_df.columns.tolist())

    # 3. Chargement (Export CSV)
    output_path = "OWID_Mental_Health_Burden_2021.csv"
    clean_df.to_csv(output_path, index=False)
    
    print(f"\nFichier sauvegardé avec succès : {output_path}")
    print("Prêt pour l'analyse sur VS Code.")

--- Démarrage du scraping OWID ---
Requête vers : https://ourworldindata.org/grapher/burden-of-disease-from-each-category-of-mental-illness.csv


HTTPError: 404 Client Error: Not Found for url: https://ourworldindata.org/grapher/burden-of-disease-from-each-category-of-mental-illness.csv

In [4]:
import requests
import pandas as pd
import io

# ---------------------------------------------------------
# CONFIGURATION
# ---------------------------------------------------------
# CORRECTION ICI : Le bon slug pour ce graphique spécifique
BASE_URL = "https://ourworldindata.org/grapher/"
CHART_SLUG = "burden-disease-from-each-mental-illness"

def get_owid_data(slug):
    """
    Récupère le fichier CSV complet associé à un graphique OWID.
    """
    url = f"{BASE_URL}{slug}.csv"
    print(f"Requête vers : {url}")
    
    resp = requests.get(url)
    resp.raise_for_status() # Lève une erreur si le lien est mauvais
    
    return pd.read_csv(io.StringIO(resp.content.decode('utf-8')))

def process_mental_health_data(df, target_year=2021):
    """
    Nettoie et filtre les données pour ne garder que l'année cible.
    """
    # 1. Filtrage temporel
    df_filtered = df[df['Year'] == target_year].copy()
    
    # 2. Suppression de la colonne année
    df_filtered = df_filtered.drop(columns=['Year'])
    
    # 3. Renommage des colonnes pour faciliter les régressions (retirer espaces et parenthèses)
    # Ex: "Anxiety disorders (DALYs)" -> "Anxiety_disorders"
    df_filtered.columns = [
        col.replace(' (DALYs)', '')
           .replace(' ', '_')
           .replace('-', '_')
        for col in df_filtered.columns
    ]
    
    # 4. On ne garde que les lignes qui ont un Code pays (pour éliminer les continents/régions)
    df_countries_only = df_filtered.dropna(subset=['Code'])
    
    return df_countries_only

# ---------------------------------------------------------
# PARTIE PRINCIPALE
# ---------------------------------------------------------

if __name__ == "__main__":
    
    print("--- Démarrage du scraping OWID (Corrigé) ---")

    try:
        # 1. Extraction
        raw_df = get_owid_data(CHART_SLUG)
        print(f"Données brutes récupérées : {raw_df.shape[0]} lignes.")

        # 2. Transformation
        clean_df = process_mental_health_data(raw_df, target_year=2021)
        print(f"Données filtrées (2021, pays uniquement) : {clean_df.shape[0]} lignes.")
        
        # 3. Chargement (Export au bon endroit)
        # J'ai mis le chemin absolu pour que ce soit rangé avec tes données OMS
        output_path = "/home/onyxia/python-DATA-1/Données_OMS/OWID_Mental_Health_Burden_2021.csv"
        
        clean_df.to_csv(output_path, index=False)
        
        print(f"\nSUCCÈS ! Fichier sauvegardé ici : {output_path}")
        print("Colonnes prêtes pour l'analyse :", clean_df.columns.tolist())

    except Exception as e:
        print(f"\nErreur rencontrée : {e}")

--- Démarrage du scraping OWID (Corrigé) ---
Requête vers : https://ourworldindata.org/grapher/burden-disease-from-each-mental-illness.csv

Erreur rencontrée : 403 Client Error: Forbidden for url: https://ourworldindata.org/grapher/burden-disease-from-each-mental-illness.csv


In [6]:
import requests
import pandas as pd
import io

# ---------------------------------------------------------
# CONFIGURATION
# ---------------------------------------------------------
BASE_URL = "https://ourworldindata.org/grapher/"
CHART_SLUG = "burden-disease-from-each-mental-illness"

# Chemin de sortie (le même que pour tes données OMS)
OUTPUT_PATH = "/home/onyxia/python-DATA-1/Données_OMS/OWID_Mental_Health_Burden_2021.csv"

def get_owid_data(slug):
    """
    Récupère le fichier CSV en se faisant passer pour un navigateur.
    """
    url = f"{BASE_URL}{slug}.csv"
    print(f"Requête vers : {url}")
    
    # C'EST ICI QUE LA MAGIE OPÈRE :
    # On définit un User-Agent pour éviter l'erreur 403.
    # Cela fait croire au site qu'on est un utilisateur sur Windows utilisant Chrome.
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    
    # On ajoute l'argument headers=headers
    resp = requests.get(url, headers=headers)
    
    # Si ça échoue encore, on aura le détail de l'erreur
    resp.raise_for_status()
    
    return pd.read_csv(io.StringIO(resp.content.decode('utf-8')))

def process_mental_health_data(df, target_year=2021):
    """
    Nettoie et filtre les données pour ne garder que l'année cible.
    """
    # 1. Filtrage pour 2021
    df_filtered = df[df['Year'] == target_year].copy()
    
    # 2. Suppression de la colonne année inutile
    df_filtered = df_filtered.drop(columns=['Year'])
    
    # 3. Renommage des colonnes (nettoyage des parenthèses et espaces)
    df_filtered.columns = [
        col.replace(' (DALYs)', '')
           .replace(' ', '_')
           .replace('-', '_')
        for col in df_filtered.columns
    ]
    
    # 4. On garde uniquement les entités qui ont un Code ISO (les pays)
    df_countries_only = df_filtered.dropna(subset=['Code'])
    
    return df_countries_only

# ---------------------------------------------------------
# MAIN
# ---------------------------------------------------------

if __name__ == "__main__":
    
    print("--- Démarrage du scraping OWID (Mode 'User-Agent') ---")

    try:
        # 1. Extraction
        raw_df = get_owid_data(CHART_SLUG)
        print(f"Connexion réussie ! Données brutes : {raw_df.shape[0]} lignes.")

        # 2. Transformation
        clean_df = process_mental_health_data(raw_df, target_year=2021)
        print(f"Données filtrées (2021, pays uniquement) : {clean_df.shape[0]} lignes.")
        
        # 3. Chargement
        clean_df.to_csv(OUTPUT_PATH, index=False)
        
        print(f"\nSUCCÈS TOTAL ! Fichier sauvegardé ici :")
        print(OUTPUT_PATH)
        print("\nPremières colonnes disponibles :")
        print(clean_df.columns.tolist()[:5]) # Affiche les 5 premières pour vérifier

    except Exception as e:
        print(f"\nUne erreur est survenue : {e}")

--- Démarrage du scraping OWID (Mode 'User-Agent') ---
Requête vers : https://ourworldindata.org/grapher/burden-disease-from-each-mental-illness.csv

Une erreur est survenue : 403 Client Error: Forbidden for url: https://ourworldindata.org/grapher/burden-disease-from-each-mental-illness.csv


In [7]:
import requests
import pandas as pd
import io

# ---------------------------------------------------------
# CONFIGURATION
# ---------------------------------------------------------
BASE_URL = "https://ourworldindata.org/grapher/"
CHART_SLUG = "burden-disease-from-each-mental-illness"
OUTPUT_PATH = "/home/onyxia/python-DATA-1/Données_OMS/OWID_Mental_Health_Burden_2021.csv"

def get_owid_data(slug):
    """
    Récupère le CSV en simulant parfaitement une navigation humaine.
    """
    # On ajoute ?v=1 pour simuler le cache browser, parfois ça aide
    url = f"{BASE_URL}{slug}.csv?v=1" 
    
    print(f"Tentative de connexion vers : {url}")
    
    # HEADERS COMPLETS : On imite Chrome à 100%
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
        "Referer": f"https://ourworldindata.org/grapher/{slug}", # Très important : dire qu'on vient de la page du graphique
        "Accept": "text/csv,text/plain;q=0.9,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.9",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "same-origin",
        "Connection": "keep-alive"
    }
    
    # On utilise une 'Session' pour gérer les cookies si nécessaire
    session = requests.Session()
    session.headers.update(headers)
    
    resp = session.get(url)
    
    # Gestion d'erreur explicite
    if resp.status_code == 403:
        raise Exception("Accès refusé (403). Le site bloque l'IP du serveur Onyxia.")
    elif resp.status_code == 404:
        raise Exception("Page non trouvée (404). L'URL est incorrecte.")
        
    resp.raise_for_status()
    
    return pd.read_csv(io.StringIO(resp.content.decode('utf-8')))

def process_mental_health_data(df, target_year=2021):
    # 1. Filtrage 2021
    df_filtered = df[df['Year'] == target_year].copy()
    
    # 2. Suppression Année
    df_filtered = df_filtered.drop(columns=['Year'])
    
    # 3. Nettoyage colonnes
    df_filtered.columns = [
        col.replace(' (DALYs)', '').replace(' ', '_').replace('-', '_')
        for col in df_filtered.columns
    ]
    
    # 4. Garder les pays seulement
    df_countries_only = df_filtered.dropna(subset=['Code'])
    
    return df_countries_only

# ---------------------------------------------------------
# MAIN
# ---------------------------------------------------------
if __name__ == "__main__":
    print("--- Scraping OWID (Mode Avancé) ---")

    try:
        # 1. Extraction
        raw_df = get_owid_data(CHART_SLUG)
        print(f"✅ Données téléchargées : {raw_df.shape[0]} lignes.")

        # 2. Transformation
        clean_df = process_mental_health_data(raw_df)
        print(f"✅ Données filtrées : {clean_df.shape[0]} lignes.")
        
        # 3. Chargement
        clean_df.to_csv(OUTPUT_PATH, index=False)
        print(f"✅ Fichier sauvegardé : {OUTPUT_PATH}")

    except Exception as e:
        print(f"\n❌ ERREUR FATALE : {e}")
        print("-" * 50)
        print("SOLUTION DE SECOURS SI L'ERREUR PERSISTE :")
        print("1. Ouvre ce lien sur ton PC : https://ourworldindata.org/grapher/burden-disease-from-each-mental-illness.csv")
        print("2. Le fichier va se télécharger.")
        print(f"3. Fais un glisser-déposer du fichier dans VS Code vers le dossier : /Données_OMS/")

--- Scraping OWID (Mode Avancé) ---
Tentative de connexion vers : https://ourworldindata.org/grapher/burden-disease-from-each-mental-illness.csv?v=1

❌ ERREUR FATALE : Accès refusé (403). Le site bloque l'IP du serveur Onyxia.
--------------------------------------------------
SOLUTION DE SECOURS SI L'ERREUR PERSISTE :
1. Ouvre ce lien sur ton PC : https://ourworldindata.org/grapher/burden-disease-from-each-mental-illness.csv
2. Le fichier va se télécharger.
3. Fais un glisser-déposer du fichier dans VS Code vers le dossier : /Données_OMS/
